<a href="https://colab.research.google.com/github/keptzin/DLA_absorption/blob/main/DLA_DR7_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from kevlib import common_import_script
from kevlib.utils.base_defs import robust_fit
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from matplotlib import pyplot as plt
from kevlib.io.sdss import load_sdss_spectrum
from kevlib.utils.base_defs import robust_fit

from absorber_utils_v4 import *


print("finished!")

In [ ]:
catalog = np.loadtxt("../dla_ml/data_dr9/DR9DLAcatalog.txt")

In [ ]:
number = 15
for i in range(1, number):

    robust_fit_kwargs = dict(
        ubm = 20,
        sdms = np.linspace(15, 0.2, 20),
        smooth_strength = 0.9,
        pdeg = 4,
        fit_type = 'ss',
        plot_q = False,
    )

    plate = catalog[:,1][i]
    fiber = catalog[:,3][i]
    redshift = catalog[:,4][i]

    spectrum = SDSS.get_spectra(plate = plate, fiberID = fiber)
    data = spectrum[0][1].data
    wavelength = 10 ** data['loglam']
    wavelength = wavelength / (redshift + 1)
    flux = data['flux']

    cut = (wavelength > 1195) & (wavelength < 1235)
    wavelength = wavelength[cut]
    flux = flux[cut]

    __, flux_continuum, __ = robust_fit(wavelength, flux, ** robust_fit_kwargs)

    flux = flux / flux_continuum

    plt.figure()
    plt.title("Damped Lyman Alpha DR7 ")
    plt.plot(wavelength, flux, color = "black", linewidth = 0.4, label = "plate: " + str(plate) + ", fiber: " + str(fiber) + ", Z: " + str(redshift))
    plt.legend()
    plt.axvline(1215, color = "pink")